In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mlxtend

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

**Testing Notes:** Since this file is for version 1 model consistency evaluation, we don't need to manually make the inputs to the model function except the 'folder_contains_hashtagstweets_with_sentiment_compoundvalue' variable.

We have runned this file on Google Colab, so the folder path format is associated with Google Drive. In order to test this file, please change the path variables accordingly in the following cell. For ease of testing, we have the path variable 'folder_contains_hashtagstweets_with_sentiment_compoundvalue' as one of the input for the model function, but the actual inputs are the three we mentioned in the report.

Please run every cell to the end to see the output.

In [ ]:
import os

# the path variables for required folders

# the directory that contains all the sub folders or files
main_folder = '/content/drive/MyDrive/CMPT733_Final_Project'

# the folder that contains the tables of tweets compound values for each hashtag in the hashtags list after sentiment analysis 
folder_contains_hashtagstweets_with_sentiment_compoundvalue = 'tweet_sentiment_added_noise_removed'

# the folder that contains the collected small amount of tweets of the pretended target hashtags which are described in section 5.2 of the report
folder_contains_pretended_target_hashtag_tweets = 'evaluation_testing_files/testing_tweets_data'

# the folder to store the tables that contain ground truth and prediction results for version 1 which will be used in 
# the 'Prediction_Consistency_Evaluation_V1' file
folder_store_ground_truth_prediction = 'evaluation_testing_files/v1_predicted_files'
os.chdir(main_folder)


In [ ]:
def extract_tags(Text):
  tags = re.findall(r"#(\w+)", Text)
  return tags

def find_related_tags(target_tag, target_tag_tweets):

  hashtags = ['coronavirus',
              'coronaupdate',
              'selfisolating',
              'quarantine',
              'wearamask',
              'stayhomestaysafe',
              'pneumonia',
              'herdimmunity',
              'wfh',
              'masks4all',
              'covid19',
              'faceshield',
              'sarscov2',
              'frontlineheroes',
              'washyourhands',
              'covid-19',
              'coronavaccine',
              'flattenthecurve',
              'covidvaccine',
              'workfromhome',
              'socialbubble',
              'ppe',
              'socialdistancing',
              'lockdown',
              'pandemic']

  df = pd.read_csv('./' + folder_contains_pretended_target_hashtag_tweets + '/' + target_tag_tweets, lineterminator='\n')

  df['Tags'] = df['Text'].apply(extract_tags)
  tags_list = df['Tags'].to_list()

  tags_filter_list = []
  for sublist in tags_list:
    sublist = [i for i in sublist if i in hashtags]
    if sublist:
      tags_filter_list.append(sublist)
    
  encoder = TransactionEncoder()

  tags_array_encoded = encoder.fit(tags_filter_list).transform(tags_filter_list)

  tags_df_encoded = pd.DataFrame(tags_array_encoded, columns=encoder.columns_)

  tags_cluster = apriori(tags_df_encoded, min_support=0.001, use_colnames=True)

  # remove tags contains only 1 item
  tags_cluster['length'] = tags_cluster['itemsets'].apply(lambda x: len(x))
  tags_cluster_filtered = tags_cluster[(tags_cluster['length'] >= 2)].sort_values(by=['support'], ascending=False)
  tag_list = [list(i) for i in tags_cluster_filtered.itemsets]
  tag_list

  # find related tags
  related_tags = []
  for tags in tag_list:
    if target_tag in tags:
      related_tags.extend(tags)
  related_tags = list(set(related_tags))
  related_tags.remove(target_tag)
  related_tags = [ '#'+tag for tag in related_tags]

  return related_tags


In [ ]:
# get all values
def get_average_sentiment_score(data_folder):
  files = [file for file in os.listdir(data_folder) if "tag_#" in file]

  colnames = ['date']
  daily_mean_df = pd.DataFrame(columns=['date'])
  weekly_mean_df = pd.DataFrame(columns=['date'])


  for file in files: 
      df = pd.read_csv('./' + folder_contains_hashtagstweets_with_sentiment_compoundvalue + '/' + file, lineterminator='\n')
      tag = re.sub('.csv', '', re.sub('tag_', '', file))
      colnames.append(tag)
      
      mean_df = df.groupby("date")['compound'].mean()
      mean_df.index = pd.to_datetime(mean_df.index)

      daily_mean_df = daily_mean_df.merge(mean_df, how='outer', on='date')
      weekly_mean_df = weekly_mean_df.merge(mean_df.resample("W").mean(), how='outer', on='date')
      
  daily_mean_df.columns = colnames
  weekly_mean_df.columns = colnames
  
  return daily_mean_df, weekly_mean_df

In [ ]:
# predict target tag value
def predict_target_tag(target_tag, target_tag_tweets, data_folder):
  related_tags = find_related_tags(target_tag, target_tag_tweets)
  daily_mean_df, weekly_mean_df = get_average_sentiment_score(data_folder)

  column_selected = related_tags + ['date']
  predicted_df = daily_mean_df[column_selected]
  predicted_df[target_tag] = daily_mean_df[related_tags].mean(axis=1)

  daily_mean_df['#'+target_tag] = np.where(daily_mean_df['#'+target_tag] >= 0.20, 1, daily_mean_df['#'+target_tag])
  daily_mean_df['#'+target_tag] = np.where(daily_mean_df['#'+target_tag] <= 0.15, -1, daily_mean_df['#'+target_tag]) 
  daily_mean_df['#'+target_tag] = np.where(daily_mean_df['#'+target_tag].between(0.15, 0.20), 0, daily_mean_df['#'+target_tag])

  predicted_df[target_tag] = np.where(predicted_df[target_tag] >= 0.20, 1, predicted_df[target_tag])
  predicted_df[target_tag] = np.where(predicted_df[target_tag] <= 0.15, -1, predicted_df[target_tag])
  predicted_df[target_tag] = np.where(predicted_df[target_tag].between(0.15, 0.20), 0, predicted_df[target_tag])

  target_predicted_eval = predicted_df[[target_tag]]
  target_predicted_eval[target_tag + '_ground_truth'] = daily_mean_df['#'+target_tag]
  target_predicted_eval = target_predicted_eval.dropna()
  target_predicted_eval.to_csv('./' + folder_store_ground_truth_prediction + '/' + target_tag + '_predicted_df_v1.csv')

  return target_predicted_eval

In [ ]:
hashtags = ['coronavirus',
            'coronaupdate',
            'selfisolating',
            'quarantine',
            'wearamask',
            'stayhomestaysafe',
            'pneumonia',
            'herdimmunity',
            'wfh',
            'covid19',
            'sarscov2',
            'washyourhands',
            'coronavaccine',
            'flattenthecurve',
            'covidvaccine',
            'workfromhome',
            'ppe',
            'socialdistancing',
            'lockdown',
            'pandemic']

for hashtag in hashtags:
  a = predict_target_tag(hashtag, 'eval_tag_#' + hashtag + '.csv', folder_contains_hashtagstweets_with_sentiment_compoundvalue)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs